In [1]:
import numpy as np
import pandas as pd
import random
import time

In [2]:
flow = pd.read_excel('Flow.xlsx')
cost = pd.read_excel('Costs.xlsx')

capacity = pd.read_excel('Capacity.xlsx', header = None)
capacity.set_index(0, inplace = True)
capacity.columns = ['Flow']
capacity_threshold = capacity.loc['Large Capacity','Flow']

fixed_cost = pd.read_excel('FixedCosts.xlsx')
fixed_cost.set_index('Size', inplace = True)

In [3]:
'''Initialisation, Feasibility and Cost functions'''
def initialisation(nodes):
    # Randomly select a predefined number of nodes to act as hubs
    hubs = np.random.choice(nodes, size = number_of_hubs, replace = False)
    # Keep only spokes (non-hub nodes)
    nonhubs = np.array(list(set(nodes) - set(hubs)))
    hub_allocation = nodes.copy()
    # Assign each spoke randomly to a hub
    for i in nonhubs:
        hub_allocation[i] = np.random.choice(hubs, size = 1)
    return hub_allocation

def check_hub_capacity_flow(capacity_threshold, hub_allocation, flow):
    hubs = list(set(hub_allocation))
    # Check flow for each hub
    for hub in hubs:
        sum_flow = 0
        # For calculating the flow take into account only the hub and its connected spokes
        hub_nodes = list(np.where(hub_allocation == hub)[0])
        for hnode in hub_nodes:
            sum_flow += flow.iloc[hnode,:].sum()
        # If flow exceeds capacity label solution as infeasible
        if sum_flow > capacity_threshold:
            return 'Infeasible'
    return 'Feasible'

def cost_function(flow, cost, sol, a, nodes):
    if type(sol) != np.ndarray:
        sol = np.array(sol)
    total_cost = 0
    for i in nodes:
        for j in nodes:
            total_cost += flow.iloc[i,j] * (cost.iloc[i,sol[i]] +  a * cost.iloc[sol[i],sol[j]] + 
                                            cost.iloc[sol[j],j])

    hubs = list(set(sol))
    # For each hub check flow to assign equivalent size cost
    for hub in hubs:
        sum_flow = 0
        hub_nodes = list(np.where(sol == hub)[0])
        for hnode in hub_nodes:
            sum_flow += flow.iloc[hnode,:].sum()
        if sum_flow <= capacity.loc['Small Capacity','Flow']:
            total_cost += fixed_cost.loc['Small','Cost']
        elif sum_flow <= capacity.loc['Medium Capacity','Flow']:
            total_cost += fixed_cost.loc['Medium','Cost']
        else:
            total_cost += fixed_cost.loc['Large','Cost']
    return total_cost

In [4]:
'''Neighbourhood Structures'''

# Node to Hub x1
def structure_type_a(array):
    x = array.copy()
    hubs = list(set(x))
    select_hub = np.random.choice(hubs)
    select_hub_spokes = list(np.where(x == select_hub)[0])
    while len(select_hub_spokes) < 2:
        select_hub = np.random.choice(hubs)
        select_hub_spokes = list(np.where(x == select_hub)[0])
    possible_neighbor_hubs = select_hub_spokes.copy()
    possible_neighbor_hubs.remove(select_hub)
    new_hub = np.random.choice(possible_neighbor_hubs)
    x[select_hub_spokes] = new_hub
    
    return x

# Exchange of 1-1 Nodes between 2 Hubs 
def structure_type_b(array):
    x = array.copy()
    hubs = list(set(x))
    hubs_to_remove = []
    for hub in hubs:
        hub_spokes = list(np.where(x == hub)[0])
        if len(hub_spokes) < 2:
            hubs_to_remove.append(hub)
    if len(hubs_to_remove) > 0:
        for hub in hubs_to_remove:
            hubs.remove(hub)
    if len(hubs) < 2:
        return x
    select_hubs = np.random.choice(hubs, 2, replace = False)
    select_hub1_spokes = list(np.where(x == select_hubs[0])[0])
    select_hub2_spokes = list(np.where(x == select_hubs[1])[0])
    select_hub1_spokes.remove(select_hubs[0])
    select_hub2_spokes.remove(select_hubs[1])
    spoke_1 = np.random.choice(select_hub1_spokes)
    spoke_2 = np.random.choice(select_hub2_spokes)
    x[spoke_1] = select_hubs[1]
    x[spoke_2] = select_hubs[0]
    
    return x

# Exchange of 1 Node from a Hub to another
def structure_type_c(array):
    x = array.copy()
    hubs = list(set(x))
    select_hubs = np.random.choice(hubs, 2, replace = False)
    select_hub1_spokes = list(np.where(x == select_hubs[0])[0])
    select_hub1_spokes.remove(select_hubs[0])
    while len(select_hub1_spokes) < 1:
        select_hubs = np.random.choice(hubs, 2, replace = False)
        select_hub1_spokes = list(np.where(x == select_hubs[0])[0])
        select_hub1_spokes.remove(select_hubs[0])
    spoke_1 = np.random.choice(select_hub1_spokes)
    x[spoke_1] = select_hubs[1]
    
    return x

# Node to Hub x2
def structure_type_d(array):
    x = array.copy()
    hubs = list(set(x))
    hub_len = len(hubs)
    for i in range(2):
        select_hub = np.random.choice(hubs)
        select_hub_spokes = list(np.where(x == select_hub)[0])
        stop_index = 0
        while len(select_hub_spokes) < 2:
            select_hub = np.random.choice(hubs)
            select_hub_spokes = list(np.where(x == select_hub)[0])
            stop_index += 1
            if stop_index > hub_len:
                return x
        possible_neighbor_hubs = select_hub_spokes.copy()
        possible_neighbor_hubs.remove(select_hub)
        new_hub = np.random.choice(possible_neighbor_hubs)
        x[select_hub_spokes] = new_hub
        hubs.remove(select_hub)
    
    return x

# Exchange of all nodes between 2 Hubs
def structure_type_e(array):
    x = array.copy()
    hubs = list(set(x))
    for hub in hubs:
        hub_spokes = list(np.where(x == hub)[0])
        if len(hub_spokes) < 2:
            hubs.remove(hub)
    if len(hubs) > 1:
        select_hubs = np.random.choice(hubs, 2, replace = False)
        select_hub1_spokes = list(np.where(x == select_hubs[0])[0])
        select_hub2_spokes = list(np.where(x == select_hubs[1])[0])
        select_hub1_spokes.remove(select_hubs[0])
        select_hub2_spokes.remove(select_hubs[1])
        x[select_hub1_spokes] = select_hubs[1]
        x[select_hub2_spokes] = select_hubs[0]
    else:
        return x
    
    return x

# Cyclical change of nodes
def structure_type_f(array):
    x = array.copy()
    hubs = list(set(x))
    spokes_list = []
    for i in range(len(hubs)):
        hub_spokes = list(np.where(x == hubs[i])[0])
        hub_spokes.remove(hubs[i])
        if len(hub_spokes) < 1:
            spokes_list.append(-1)
        else:
            spokes_list.append(np.random.choice(hub_spokes))
    for i, spoke in enumerate(spokes_list):
        if (spoke != -1) & (i != len(spokes_list)-1): 
            x[spoke] = hubs[i+1]
        elif i == len(spokes_list)-1:
            x[spoke] = hubs[0]
    
    return x

structures_list = [structure_type_a, structure_type_b, structure_type_c, structure_type_d, 
                   structure_type_e, structure_type_f]

In [5]:
'''Input Variables'''
nodes = np.array(range(flow.shape[0]))
print('Number of hubs:')
number_of_hubs = int(input())
print('Alpha:')
alpha = float(input())
print('Iterations:')
iter_num = int(input())

Number of hubs:
3
Alpha:
0.2
Iterations:
2000


In [7]:
start = time.time()

'''Initial Solution'''
initial_allocation = initialisation(nodes)
feasibility = check_hub_capacity_flow(capacity_threshold, initial_allocation, flow)

while (feasibility == 'Infeasible'): 
    initial_allocation = initialisation(nodes)
    feasibility = check_hub_capacity_flow(capacity_threshold, initial_allocation, flow)

current_solution = initial_allocation.copy()
current_cost = cost_function(flow, cost, current_solution, alpha, nodes)

'''RVNS'''
iterations = list(range(iter_num))
breakout = 0
for i in iterations:
    breakout += 1
    for structure in structures_list:
        new_solution = structure(current_solution).copy()
        new_feasibility = check_hub_capacity_flow(capacity_threshold, new_solution, flow)
        while (new_feasibility == 'Infeasible'):
            new_solution = structure(current_solution)
            new_feasibility = check_hub_capacity_flow(capacity_threshold, new_solution, flow)
        new_cost = cost_function(flow, cost, new_solution, alpha, nodes)
        if new_cost < current_cost:
            current_solution = new_solution.copy()
            current_cost = new_cost
            breakout = 0
    if breakout >= 150:
        last_iter = i
        break

end = time.time()
run_time = round(end-start, 2)

print('***Results***')
print('Time required to find optimal solution: ', run_time, ' seconds.')
print('Number of iterations required to reach optimal solution:', last_iter - breakout)
print('The optimal solution / allocation of hubs and spokes is: ', list(current_solution))
print('The cost of the optimal solution is: ', current_cost)

***Results***
Time required to find optimal solution:  8.75  seconds.
Number of iterations required to reach optimal solution: 12
The optimal solution / allocation of hubs and spokes is:  [5, 5, 5, 3, 3, 5, 6, 6, 5, 6]
The cost of the optimal solution is:  645192795.5108399
